# Use PyEZ for onbox automation task

Python is great for off-box automation task with rich libraries and API. But normally for onbox automation, especially for data collection, I normally use bash script. 

Bash script is easy to build and naturally coordinate with Junos. 

We could also use Python to do onbox automation, with the help of event option.

We could use Python script as op script, event script and commit script. Here I'm only giving examples for op script and event script.

A few messy points before we start:

    1.Starting from Junos 16.1, python is supported as the scripting language for on-box scripting. 

        Need to be configured

        set system scripts language python
        
    2. Due to some of the security concerns, we are not able to enter python interactive mode from root shell. 

        root@rigel-re0:/var/home/labroot # python
        /usr/bin/python: Operation not permitted.
        
    3. But eveything has exception. LOL. If the device is running automation enhanced code, AKA, flex version, python interactive mode is working:
    
    Hostname: jtac-QFX10002-72Q-r014
    Model: qfx10002-72q
    Junos: 18.4R2-S3 flex
    
    abroot@jtac-QFX10002-72Q-r014> start shell user root 
    Password:
    root@jtac-QFX10002-72Q-r014:RE:0% python
    Python 2.7.8 (default, Dec 26 2019, 17:20:57) 
    [GCC 4.2.1 Compatible Juniper Clang 5.0.2 (git@psd-tools-git01.juniper.net:tool on junos
    Type "help", "copyright", "credits" or "license" for more information.
    >>> 
    
    
    4. for on-box scripting, we will utilize one Python library quite often -- jcs library. However, this library is not working in python interactive mode
    
    >>> import jcs
    Traceback (most recent call last):
      File "<stdin>", line 1, in <module>
    ImportError: No module named jcs
    >>> 


    5. How to run and debug the python script on box? It has to be configured as a OP script and then run from Junos cli.
    
    

Since we are talking about on-box scripting, all the python code in this notebook will NOT run. Please copy it to the Junos box and play with it.
 

# Python OP script 

First thing to do is to config the script as an OP script

set system scripts op file python_op_script.py

and put the script under /var/run/scripts/op



One simple example:


root@jtac-QFX10002-72Q-r014:RE:0% vi python_op_script.py 
#!/usr/bin/env python

print('This is a python op script')
 
print('Print supported modules')
 
print(help("modules"))


labroot@jtac-QFX10002-72Q-r014# run op python_op_script.py    
Dec 30 16:27:00
This is a python op script
Print supported modules

Please wait a moment while I gather a list of all available modules...
BaseHTTPServer
bz2
jinja2
rlcompleter
Bastion
cPickle
jnpr
robotparser
CGIHTTPServer
cProfile
jrandom
runpy
ConfigParser
cStringIO
json
sched
Cookie
calendar
junos
scp
Crypto
cffi
jxmlease
select
DocXMLRPCServer
cgi
keyword
serial
HTMLParser
cgitb
lib2to3
server-extension
IN
chunk
linecache
sets
JetHandler
cmath
locale
sgmllib
MimeWriter
cmd
logging
sha
Queue
codecs
lxml
shelve
SimpleHTTPServer
codeop
macpath
shlex
SimpleXMLRPCServer                      
collections
macurl2path
shutil
SocketServer
colorsys
mailbox
signal
StringIO
commands
mailcap
site
UserDict
compileall
markupbase
six
UserList
compiler
markupsafe
smtpd
UserString
concurrent
marshal
smtplib
_LWPCookieJar
contextlib
math
sndhdr
_MozillaCookieJar
cookielib
md5
socket
__builtin__
copy
mhlib
sqlite3
__future__
copy_reg
mimetools
sre
_abcoll
crypt
mimetypes
sre_compile
_ast
cryptography
mimify
sre_constants
_bisect
csv
mmap
sre_parse
_cffi_backend
curses
modulefinder
ssl
_codecs                                 
datetime
multifile
stat
_codecs_cn
dbhash
multiprocessing
statvfs
_codecs_hk
decimal
mutex
string
_codecs_iso2022
difflib
ncclient
stringold
_codecs_jp
dircache
netaddr
stringprep
_codecs_kr
dis
netrc
strop
_codecs_tw
distutils
new
struct
_collections
dl
nntplib
subprocess
_csv
doctest
notification
sunau
_curses
dumbdbm
ntpath
sunaudio
_elementtree
dummy_thread
nturl2path
symbol
_functools
dummy_threading
numbers
symtable
_hashlib
ecdsa
opcode
sys
_heapq
email
operator
sysconfig
_hotshot                                
encodings
optparse
syslog
_io
enum
os
tabnanny
_json
errno
os2emxpath
tarfile
_locale
exceptions
packaging
telnetlib
_lsprof
fcntl
paho
tempfile
_md5
filecmp
paramiko
termios
_multibytecodec
fileinput
parser
textwrap
_multiprocessing
fnmatch
pdb
this
_pyio
formatter
pickle
thread
_random
fpformat
pickletools
threading
_sha
fractions
pipes
thrift
_sha256
ftplib
pkg_resources
time
_sha512
functools
pkgutil
timeit
_socket
future_builtins
platform
toaiff
_sre                                    
gc
plistlib
token
_ssl
genericpath
popen2
tokenize
_strptime
getopt
poplib
trace
_struct
getpass
posix
traceback
_symtable
gettext
posixfile
tty
_sysconfigdata
glob
posixpath
types
_testcapi
google
pprint
unicodedata
_threading_local
grp
profile
unittest
_warnings
grpc
pstats
urllib
_weakref
gzip
psutil
urllib2
_weakrefset
hashlib
pty
urlparse
abc
heapq
pwd
uu
aifc
hmac
py_compile
uuid
antigravity
hotshot
pyang
warnings
anydbm                                  
htmlentitydefs
pyasn1
wave
appdirs
htmllib
pyclbr
weakref
argparse
httplib
pycparser
webbrowser
array
idlelib
pydoc
whichdb
asn1crypto
idna
pydoc_data
wsgiref
ast
ihooks
pyexpat
xdrlib
asynchat
imaplib
pyparsing
xml
asyncore
imghdr
python_op_script
xmllib
atexit
imp
quopri
xmlrpclib
audiodev
importlib
random
xxsubtype
base64
imputil
re
yaml
bdb
inspect
readline
zipfile
binascii
interface_status_check
repr
zipimport
binhex
io
resource
zlib
bisect                                  
ipaddress
rexec
bsddb185
itertools
rfc822

Enter any module name to get more help.  Or, type "modules spam" to search
for modules whose descriptions contain the word "spam".
None


Most of the commom library are there.



check the supported python version:


root@jtac-QFX10002-72Q-r014:RE:0% vi python_op_script.py
#!/usr/bin/env python
import platform


print('This is a python op script')

#print('Print supported modules')

#print(help("modules"))
 
 
print(platform.sys.version)




labroot@jtac-QFX10002-72Q-r014# run op python_op_script.py    
Dec 30 16:33:58
This is a python op script
2.7.8 (default, Dec 26 2019, 17:20:57) 
[GCC 4.2.1 Compatible Juniper Clang 5.0.2 (git@psd-tools-git01.juniper.net:tool





# # print log into messages file

root@jtac-QFX10002-72Q-r014:RE:0% vi python_op_script.py
#!/usr/bin/env python

import jcs

print('This is a python op script')
print('Printing logs in messages')

jcs.syslog('user.err','Print log to log messages')

labroot@jtac-QFX10002-72Q-r014> op python_op_script.py    
This is a python op script
Printing logs in messages

Dec 30 19:05:46  jtac-QFX10002-72Q-r014 cscript: Print log to log messages


# argparse in op script

root@jtac-QFX10002-72Q-r014:RE:0% vi python_op_script.py
#!/usr/bin/env python

import jcs
import argparse


print('This is a python op script')
print('Op script  arg parser')

parser = argparse.ArgumentParser()


parser.add_argument('-arg1')
parser.add_argument('-arg2')
 
 
args = parser.parse_args()
 
 
print('Taking two args from Junos cli: {}  {}'.format(args.arg1,args.arg2))


{master:0}
labroot@jtac-QFX10002-72Q-r014> op python_op_script.py arg1 OSPF arg2 BGP 
This is a python op script
Op script  arg parser
Taking two args from Junos cli: OSPF  BGP





# Interact with CLi and shell

root@jtac-QFX10002-72Q-r014:RE:0% vi python_op_script.py
#!/usr/bin/env python

import subprocess
 
print('Op script interact with cli command')
 
print('show version\n')
cmd='cli -c "show version"'
output=subprocess.check_output(cmd,shell=True)
print(output)



{master:0}
labroot@jtac-QFX10002-72Q-r014> op python_op_script.py                                            
Op script interact with cli command
show version
Hostname: jtac-QFX10002-72Q-r014
Model: qfx10002-72q
Junos: 18.4R2-S3 flex
JUNOS OS Kernel 64-bit FLEX [20191223.5f5c7dc_builder_stable_11]
JUNOS OS libs [20191223.5f5c7dc_builder_stable_11]
JUNOS OS runtime [20191223.5f5c7dc_builder_stable_11]
JUNOS OS time zone information [20191223.5f5c7dc_builder_stable_11]
JUNOS OS libs compat32 [20191223.5f5c7dc_builder_stable_11]
JUNOS OS 32-bit compatibility [20191223.5f5c7dc_builder_stable_11]
JUNOS py extensions [20191226.165110_builder_junos_184_r2_s3]
JUNOS py base [20191226.165110_builder_junos_184_r2_s3]
JUNOS OS vmguest [20191223.5f5c7dc_builder_stable_11]
JUNOS OS crypto [20191223.5f5c7dc_builder_stable_11]
JUNOS network stack and utilities [20191226.165110_builder_junos_184_r2_s3]
JUNOS libs [20191226.165110_builder_junos_184_r2_s3]
JUNOS libs compat32 [20191226.165110_builder_junos_184_r2_s3]
JUNOS runtime [20191226.165110_builder_junos_184_r2_s3]
JUNOS na telemetry [18.4R2-S3]
JUNOS Web Management Platform Package [20191226.165110_builder_junos_184_r2_s3]
JUNOS qfx runtime [20191226.165110_builder_junos_184_r2_s3]
JUNOS common platform support [20191226.165110_builder_junos_184_r2_s3]
JUNOS qfx platform support [20191226.165110_builder_junos_184_r2_s3]
JUNOS Openconfig [18.4R2-S3]
JUNOS dcp network modules [20191226.165110_builder_junos_184_r2_s3]
JUNOS modules [20191226.165110_builder_junos_184_r2_s3]
JUNOS qfx modules [20191226.165110_builder_junos_184_r2_s3]
JUNOS qfx Data Plane Crypto Support [20191226.165110_builder_junos_184_r2_s3]
JUNOS daemons [20191226.165110_builder_junos_184_r2_s3]
JUNOS qfx daemons [20191226.165110_builder_junos_184_r2_s3]
JUNOS SDN Software Suite [20191226.165110_builder_junos_184_r2_s3]
JUNOS Extension Toolkit [20191226.165110_builder_junos_184_r2_s3]
JET app jpuppet [3.6.1_4.0]
JUNOS Packet Forwarding Engine Support (DC-PFE) [20191226.165110_builder_junos_184_r2_s3]
JUNOS Packet Forwarding Engine Support (M/T Common) [20191226.165110_builder_junos_184_r2_s3]
JUNOS J-Insight [20191226.165110_builder_junos_184_r2_s3]
JUNOS jfirmware [20191226.165110_builder_junos_184_r2_s3]
JUNOS Online Documentation [20191226.165110_builder_junos_184_r2_s3]
JUNOS jail runtime [20191223.5f5c7dc_builder_stable_11]
JET app chef [11.10.4_3.0]
JUNOS Host Software [3.14.64-rt67-WR7.0.0.26_ovp:3.1.0]
JUNOS Host qfx-10-f platform package [18.4R2-S3]
JUNOS Host qfx-10-f data-plane package [18.4R2-S3]
JUNOS Host qfx-10-f base package [18.4R2-S3]
JUNOS Host qfx-10-f control-plane flex package [18.4R2-S3]
Junos for Automation Enhancement


# OP script with PyEZ

labroot@jtac-QFX10002-72Q-r014> op python_op_script.py    
<fpc-information style="brief">
<fpc>
<slot>0</slot>
<state>Online</state>
<temperature celsius="25">25</temperature>
<cpu-total>4</cpu-total>
<cpu-interrupt>0</cpu-interrupt>
<cpu-1min-avg>3</cpu-1min-avg>
<cpu-5min-avg>3</cpu-5min-avg>
<cpu-15min-avg>3</cpu-15min-avg>
<memory-dram-size>16384</memory-dram-size>
<memory-heap-utilization>24</memory-heap-utilization>
<memory-buffer-utilization>64</memory-buffer-utilization>
</fpc>
</fpc-information>
None
('0', 'Online', '25', '4')


from lxml import etree
from jnpr.junos import Device

with Device() as dev:

    fpcs_xpath=dev.rpc.get_fpc_information()
    print(etree.dump(fpcs_xpath))
    fpcs=fpcs_xpath.iter()
    for fpc in fpcs:
        if fpc.tag=='fpc':
            print(fpc.findtext('slot'), fpc.findtext('state'),fpc.findtext('temperature'),fpc.findtext('cpu-total'))
~

# Collect data from cli and shell with python op script

root@jtac-QFX10002-72Q-r014:RE:0% vi python_op_script.py
#!/usr/bin/env python
from lxml import etree
import jcs
import datetime
from jnpr.junos import Device
import subprocess

def cli_data(dev):

    with open('/var/tmp/cli-cmds.txt','r') as f1:
        cmds=f1.readlines()
        for cmd in cmds:
            output=etree.tostring(dev.rpc.cli(command=cmd))
            file_name = '/var/tmp/cli-data-collection/{}_{}'.format(cmd.strip(), datetime.datetime.utcnow().strftime(
                '%Y%m%d_%H%M%S')).replace(' ', '_')
            write_to_file(cmd,output,file_name)


def shell_data(dev):

    with open('/var/tmp/shell-cmds.txt','r') as f1:
        cmds=f1.readlines()
        for cmd in cmds:
            output=subprocess.check_output(cmd,shell=True)
            file_name = '/var/tmp/shell-data-collection/{}_{}'.format(cmd.strip(), datetime.datetime.utcnow().strftime(
                '%Y%m%d_%H%M%S')).replace(' ', '_')
            write_to_file(cmd,output,file_name)






def write_to_file(cmd,data,file_name):



    with open(file_name,'a') as f:
        f.write(data)
        f.write('\n\n')


if __name__ =='__main__':
    with Device() as dev:
        jcs.syslog('user.err','Start the data collection script!')
        dev.rpc.make_directory(directory='/var/tmp/cli-data-collection')

        cli_data(dev)
        dev.rpc.make_directory(directory='/var/tmp/shell-data-collection')
        shell_data(dev)

# Python Event Script

Similar to OP script, Event script requires Junos configuration to setup the trigger, internal and executed script.

Here is a simple example:

This is a simple HOW-TO example to monitor interface status, collect some command output upon interface flapping, and then write the output into a file locally.


Event option referrence:
https://www.juniper.net/documentation/en_US/junos/topics/task/configuration/junos-script-automation-event-script-arguments-configuring-and-using.html

Junos configuration:
    
labroot@jtac-QFX10002-72Q-r014> show configuration event-options 
policy interface_status_check {
    events snmp_trap_link_down;
    attributes-match {
        snmp_trap_link_down.interface-name matches xe-;
    }
    then {
        event-script interface_status_check.py {
            arguments {
                interface "{$$.interface-name}";
            }
        }
    }
}
event-script {
    optional;
    file interface_status_check.py {
        python-script-user labroot;
    }
}


labroot@jtac-QFX10002-72Q-r014> file show /var/run/scripts/event/interface_status_check.py 
#!/usr/bin/env python
 
import argparse
 
from jnpr.junos import Device
import jcs
import datetime
from lxml import etree
import sys
import platform
 
 
 
 
 
 
if __name__=='__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-interface')
    args = parser.parse_args()
    jcs.syslog('user.err','Python version {}'.format(platform.sys.version))
    int_name=args.interface
    with Device() as dev:
        jcs.syslog('user.err','{} goes down, start the data collection!!!!!'.format(int_name))
 
        with open('/var/tmp/cmds.txt','r') as cmd_file:
            cmds=cmd_file.readlines()
            for cmd in cmds:
                jcs.syslog('user.err','Collecting {}\n'.format(cmd))
                output=etree.tostring(dev.rpc.cli(command=cmd))
 
                file_name='/var/tmp/{}_{}'.format(cmd.strip(),datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')).replace(' ','_')
                with open(file_name,'a') as f:
                    f.write(output)
                    f.write('\n\n')
 
 

        

Above script achieve some basic function:
    
    Monotor for interface down event.
    Once a xe interface goes down, collect some command in a seperate cmds.txt file
    Save the command output under /var/tmp with the command name and timestamp as filename.
    
    


For event script, there is no log printed on the cli, so it is not convenient to debug the code. 

To determine whether the script is triggered successfully, please check the escript.log file under /var/log

